In [2]:
import pandas
import numpy
import math
%run Punto.ipynb
%run Circunferencia.ipynb
%run grafico.ipynb
%run grado_pertenencia.ipynb

# CLASE DE MÉTODOS PARA EL ALGORITMO K-MEDIAS

### Método de inicialización

##### Inicializa las circunferencias iniciales, posicionándolas cerca del punto con el que tienen mayor preferencia.

In [ ]:
def inicializar_circunferencias(mapa, cs):
    circunferencias = []
    l = {}
        
    for i in range(0, len(cs)):
        mx = 0
        for p, lp in mapa.items():
            if lp[0] == i and lp[1] > mx:
                mx = lp[1]
                l[i] = p
    
    for i in range(0, len(l)):
        if i in l.keys():
            cs[i].centro = Punto(l.get(i).x - 1., l.get(i).y - 1.)
        
    circunferencias = cs[:]
    return circunferencias

### Métodos de clusters

##### Crea los clústeres en cada iteración, la lista de circunferencias se actualiza en cada iteración y por tanto los clústeres también

In [2]:
def crear_clusters(mapa,circunferencias):
    
    puntos = list(mapa.keys())
    listas_grados_indice = list(mapa.values())
    lista_clusters = []
    
    for c in circunferencias:
        cl=Cluster(c,[])
        lista_clusters.append(cl)
        
    lista_clusters = obtener_maxima_pertenencia(mapa, lista_clusters)
    
    return lista_clusters  ## Devuelve los clusters iniciales

##### Estos métodos actualizan un cluster determinado, en cada iteración actualizan la lista de clústeres clúster a clúster para calcular las nuevas circunferencias

In [4]:
def actualizar_radio(cluster):
    N = len(cluster.puntos)
    dist = 0
    if cluster.circunferencia.radio == 0 or N == 0 or N == 1:
        return numpy.random.randint(1,3)
    else:
        for p in cluster.puntos:
            dist += cluster.circunferencia.centro.distancia(p)
        nuevoRadio = round(dist/N, 2)
        return nuevoRadio

In [5]:
def actualizar_centro(cluster):
    N = len(cluster.puntos)
    x=0
    y=0
    if N == 0:
        x = numpy.random.randint(1,10)
        y = numpy.random.randint(1,10)
        centro=Punto(x,y)
        return centro
    else:
        for p in cluster.puntos:
            x += p.x
            y += p.y
            nuevoCentro = Punto(round(x/N, 1), round(y/N, 1))
        return nuevoCentro

In [17]:
def actualizar_cluster(cluster, circunferencias, idC, mapa):
    nuevoCentro = actualizar_centro(cluster)
    puntos = list(mapa.keys())
    
    C = Circunferencia(nuevoCentro,cluster.circunferencia.radio) ##Actualizamos el centro y sin actualizar el radio volvemos 
    circunferencias[idC] = C                                     ##a calcular el grado de pertenencia con todos los puntos
    
    nuevo_mapa = grado_pertenencia(puntos, circunferencias)      
    
    cl = crear_clusters(nuevo_mapa, circunferencias)
    
    nuevoRadio = actualizar_radio(cl[idC])
    
    resC = Circunferencia(nuevoCentro,nuevoRadio)
    resCl = Cluster(resC,cl[idC].puntos)
    
    circunferencias[idC] = resC
    
    return resCl

### Método para el criterio de parada

##### Comprueba antes de seguir con la siguiente iteración, si el algoritmo ha encontrado un resultado válido para devolver

In [18]:
def circunferencias_iguales(cls,cls2,mapa, umbral):
    b = True
    
    for i in range(0,len(cls)):
        if cls2[i].puntos == [] or len(cls2[i].puntos) == 1:
            b = False
            break
        if not cls2[i].circunferencia.equals(cls[i].circunferencia):
            b = False
            break
        else:
            for p in range(0,len(cls2[i].puntos)):
                if not todos_los_grados_maximizados(cls2[i], mapa, i, umbral):
                    b = False
                    break
    return b

## Algoritmo k-medias

In [ ]:
def k_medias(puntos,circunferencias, it, umbral):
    mapa=grado_pertenencia(puntos,circunferencias)
    circunferencias = inicializar_circunferencias(mapa, circunferencias)
    i = 0
    while i < it:
        mapa=grado_pertenencia(puntos,circunferencias)
        cls = crear_clusters(mapa,circunferencias)
        clustersActualizados = []
        circunferenciasActualizadas = []
        sorted(cls)
        for cl in cls:
            clId = cls.index(cl)
            nuevoCluster = actualizar_cluster(cl, circunferencias, clId, mapa)
            clustersActualizados.append(nuevoCluster)
            circunferenciasActualizadas.append(nuevoCluster.circunferencia)
        nuevo_mapa = grado_pertenencia(puntos,circunferenciasActualizadas)
        mapa = nuevo_mapa
        if circunferencias_iguales(cls,clustersActualizados,mapa, umbral):
            break;
        cls = clustersActualizados[:]
        sorted(cls)
        circunferencias = circunferenciasActualizadas[:]
        i += 1
    return cls